# **Import libraries**

In [24]:
!pip install mido

In [25]:
import mido # easy to use python MIDI library
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import random
import pandas as pd

from mido import MidiFile, MidiTrack, Message

from sklearn import model_selection

import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# **Hyperparameters**

In [27]:
num_epochs = 100
batch_size = 1024

sequence_length = 16
embedding_dim_note = 128
embedding_dim_duree = 128
embedding_dim_time = 128
embedding_dim_veloc = 128

hidden_size = 512
num_layers = 3
num_classes_note = 128
num_classes_duree = 32
num_classes_time = 18

learning_rate = 0.05

# **Load data**

In [28]:
df_train = pd.read_csv('train_all.csv', header=None)
df_val = pd.read_csv('val_all.csv', header=None)

array_train = df_train.values
array_val = df_val.values

In [29]:
train_loader = torch.utils.data.DataLoader(dataset=array_train,
                                           batch_size=batch_size, 
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(dataset=array_val,
                                           batch_size=batch_size, 
                                           shuffle=True)

# **Models**

## RNN

In [38]:
# Recurrent neural network (many-to-one)
class RNN(nn.Module):
  def __init__(self, num_classes_note, num_classes_duree, num_classes_time, embedding_dim_note, embedding_dim_duree, embedding_dim_time, embedding_dim_veloc, hidden_size, num_layers, drop_prob=0., drop_fc=0.):
    super(RNN, self).__init__()

    self.embedding_note = nn.Embedding(num_classes_note, embedding_dim_note)
    self.embedding_duree = nn.Embedding(num_classes_duree, embedding_dim_duree)
    self.embedding_time = nn.Embedding(num_classes_time, embedding_dim_time)
    self.fc_emb_veloc = nn.Linear(1, embedding_dim_veloc)

    self.embedding_note_cond = nn.Embedding(num_classes_note, embedding_dim_note)
    self.embedding_duree_cond = nn.Embedding(num_classes_duree, embedding_dim_duree)
    self.embedding_time_cond = nn.Embedding(num_classes_time, embedding_dim_time)

    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.rnn = nn.RNN(embedding_dim_note + embedding_dim_duree + embedding_dim_time + embedding_dim_veloc, hidden_size, num_layers, dropout=drop_prob, batch_first=True)
    
    self.fc_note_1 = nn.Linear(hidden_size, hidden_size)
    self.fc_note_2 = nn.Linear(hidden_size, num_classes_note)

    self.fc_duree_1 = nn.Linear(hidden_size + embedding_dim_note, hidden_size)
    self.fc_duree_2 = nn.Linear(hidden_size, num_classes_duree)

    self.fc_time_1 = nn.Linear(hidden_size + embedding_dim_note + embedding_dim_duree, hidden_size)
    self.fc_time_2 = nn.Linear(hidden_size, num_classes_time)

    self.fc_veloc_1 = nn.Linear(hidden_size + embedding_dim_note + embedding_dim_duree + embedding_dim_time, hidden_size)
    self.fc_veloc_2 = nn.Linear(hidden_size, 1)

    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=drop_fc)

  def foward_RNN(self, inputs):
    notes, duree, time, veloc = inputs

    # Embedding layer
    embeddings_note = self.embedding_note(notes) # Output shape (batch, sequence_length, embedding_dim)
    embeddings_duree = self.embedding_duree(duree)
    embeddings_time = self.embedding_time(time)

    emeddings_veloc = self.fc_emb_veloc(veloc)

    x = torch.cat((embeddings_note, embeddings_duree, embeddings_time, emeddings_veloc), dim=2)

    # Set initial hidden and cell states 
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
    
    # Forward propagate LSTM
    out, hidden = self.rnn(x, h0)  # out: tensor of shape (batch_size, seq_length, hidden_size)
    out = out[:, -1, :] # Hidden state of the last element of the sequence (equivalent to hidden[0])

    return out

  def classif_note(self, out):
    note = self.fc_note_1(out)
    note = self.dropout(note)
    note = self.relu(note)
    note = self.fc_note_2(note)
    return note

  def classif_duree(self, out, note_target):

    embeddings_note_target = self.embedding_note_cond(note_target)

    duree = torch.cat((out, embeddings_note_target), dim=1)
    duree = self.fc_duree_1(duree)
    duree = self.dropout(duree)
    duree = self.relu(duree)
    duree = self.fc_duree_2(duree)
    return duree

  def classif_time(self, out, note_target, duree_target):

    embeddings_note_target = self.embedding_note_cond(note_target)
    embeddings_duree_target = self.embedding_duree_cond(duree_target)

    time = torch.cat((out, embeddings_note_target, embeddings_duree_target), dim=1)
    time = self.fc_time_1(time)
    time = self.dropout(time)
    time = self.relu(time)
    time = self.fc_time_2(time)
    return time

  def reg_veloc(self, out, note_target, duree_target, time_target):

    embeddings_note_target = self.embedding_note_cond(note_target)
    embeddings_duree_target = self.embedding_duree_cond(duree_target)
    embeddings_time_target = self.embedding_time_cond(time_target)

    veloc = torch.cat((out, embeddings_note_target, embeddings_duree_target, embeddings_time_target), dim=1)
    veloc = self.fc_veloc_1(veloc)
    veloc = self.dropout(veloc)
    veloc = self.relu(veloc)
    veloc = self.fc_veloc_2(veloc)
    return veloc


  def forward(self, inputs, targets):
    out = self.foward_RNN(inputs)

    note_target, duree_target, time_target = targets

    note = self.classif_note(out)
    duree = self.classif_duree(out, note_target)
    time = self.classif_time(out, note_target, duree_target)
    veloc = self.reg_veloc(out, note_target, duree_target, time_target)

    return note, duree, time, veloc

## GRU

In [39]:
# Recurrent neural network (many-to-one)
class GRU(nn.Module):
  def __init__(self, num_classes_note, num_classes_duree, num_classes_time, embedding_dim_note, embedding_dim_duree, embedding_dim_time, embedding_dim_veloc, hidden_size, num_layers, drop_prob=0., drop_fc=0.):
    super(GRU, self).__init__()

    self.embedding_note = nn.Embedding(num_classes_note, embedding_dim_note)
    self.embedding_duree = nn.Embedding(num_classes_duree, embedding_dim_duree)
    self.embedding_time = nn.Embedding(num_classes_time, embedding_dim_time)
    self.fc_emb_veloc = nn.Linear(1, embedding_dim_veloc)

    self.embedding_note_cond = nn.Embedding(num_classes_note, embedding_dim_note)
    self.embedding_duree_cond = nn.Embedding(num_classes_duree, embedding_dim_duree)
    self.embedding_time_cond = nn.Embedding(num_classes_time, embedding_dim_time)

    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.rnn = nn.GRU(embedding_dim_note + embedding_dim_duree + embedding_dim_time + embedding_dim_veloc, hidden_size, num_layers, dropout=drop_prob, batch_first=True)
    
    self.fc_note_1 = nn.Linear(hidden_size, hidden_size)
    self.fc_note_2 = nn.Linear(hidden_size, num_classes_note)

    self.fc_duree_1 = nn.Linear(hidden_size + embedding_dim_note, hidden_size)
    self.fc_duree_2 = nn.Linear(hidden_size, num_classes_duree)

    self.fc_time_1 = nn.Linear(hidden_size + embedding_dim_note + embedding_dim_duree, hidden_size)
    self.fc_time_2 = nn.Linear(hidden_size, num_classes_time)

    self.fc_veloc_1 = nn.Linear(hidden_size + embedding_dim_note + embedding_dim_duree + embedding_dim_time, hidden_size)
    self.fc_veloc_2 = nn.Linear(hidden_size, 1)

    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=drop_fc)

  def foward_RNN(self, inputs):
    notes, duree, time, veloc = inputs

    # Embedding layer
    embeddings_note = self.embedding_note(notes) # Output shape (batch, sequence_length, embedding_dim)
    embeddings_duree = self.embedding_duree(duree)
    embeddings_time = self.embedding_time(time)

    emeddings_veloc = self.fc_emb_veloc(veloc)

    x = torch.cat((embeddings_note, embeddings_duree, embeddings_time, emeddings_veloc), dim=2)

    # Set initial hidden and cell states 
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
    
    # Forward propagate LSTM
    out, hidden = self.rnn(x, h0)  # out: tensor of shape (batch_size, seq_length, hidden_size)
    out = out[:, -1, :] # Hidden state of the last element of the sequence (equivalent to hidden[0])

    return out

  def classif_note(self, out):
    note = self.fc_note_1(out)
    note = self.dropout(note)
    note = self.relu(note)
    note = self.fc_note_2(note)
    return note

  def classif_duree(self, out, note_target):

    embeddings_note_target = self.embedding_note_cond(note_target)

    duree = torch.cat((out, embeddings_note_target), dim=1)
    duree = self.fc_duree_1(duree)
    duree = self.dropout(duree)
    duree = self.relu(duree)
    duree = self.fc_duree_2(duree)
    return duree

  def classif_time(self, out, note_target, duree_target):

    embeddings_note_target = self.embedding_note_cond(note_target)
    embeddings_duree_target = self.embedding_duree_cond(duree_target)

    time = torch.cat((out, embeddings_note_target, embeddings_duree_target), dim=1)
    time = self.fc_time_1(time)
    time = self.dropout(time)
    time = self.relu(time)
    time = self.fc_time_2(time)
    return time

  def reg_veloc(self, out, note_target, duree_target, time_target):

    embeddings_note_target = self.embedding_note_cond(note_target)
    embeddings_duree_target = self.embedding_duree_cond(duree_target)
    embeddings_time_target = self.embedding_time_cond(time_target)

    veloc = torch.cat((out, embeddings_note_target, embeddings_duree_target, embeddings_time_target), dim=1)
    veloc = self.fc_veloc_1(veloc)
    veloc = self.dropout(veloc)
    veloc = self.relu(veloc)
    veloc = self.fc_veloc_2(veloc)
    return veloc


  def forward(self, inputs, targets):
    out = self.foward_RNN(inputs)

    note_target, duree_target, time_target = targets

    note = self.classif_note(out)
    duree = self.classif_duree(out, note_target)
    time = self.classif_time(out, note_target, duree_target)
    veloc = self.reg_veloc(out, note_target, duree_target, time_target)

    return note, duree, time, veloc

## LSTM

In [40]:
# Recurrent neural network (many-to-one)
class LSTM(nn.Module):
  def __init__(self, num_classes_note, num_classes_duree, num_classes_time, embedding_dim_note, embedding_dim_duree, embedding_dim_time, embedding_dim_veloc, hidden_size, num_layers, drop_prob=0., drop_fc=0.):
    super(LSTM, self).__init__()

    self.embedding_note = nn.Embedding(num_classes_note, embedding_dim_note)
    self.embedding_duree = nn.Embedding(num_classes_duree, embedding_dim_duree)
    self.embedding_time = nn.Embedding(num_classes_time, embedding_dim_time)
    self.fc_emb_veloc = nn.Linear(1, embedding_dim_veloc)

    self.embedding_note_cond = nn.Embedding(num_classes_note, embedding_dim_note)
    self.embedding_duree_cond = nn.Embedding(num_classes_duree, embedding_dim_duree)
    self.embedding_time_cond = nn.Embedding(num_classes_time, embedding_dim_time)

    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.rnn = nn.LSTM(embedding_dim_note + embedding_dim_duree + embedding_dim_time + embedding_dim_veloc, hidden_size, num_layers, dropout=drop_prob, batch_first=True)
    
    self.fc_note_1 = nn.Linear(hidden_size, hidden_size)
    self.fc_note_2 = nn.Linear(hidden_size, num_classes_note)

    self.fc_duree_1 = nn.Linear(hidden_size + embedding_dim_note, hidden_size)
    self.fc_duree_2 = nn.Linear(hidden_size, num_classes_duree)

    self.fc_time_1 = nn.Linear(hidden_size + embedding_dim_note + embedding_dim_duree, hidden_size)
    self.fc_time_2 = nn.Linear(hidden_size, num_classes_time)

    self.fc_veloc_1 = nn.Linear(hidden_size + embedding_dim_note + embedding_dim_duree + embedding_dim_time, hidden_size)
    self.fc_veloc_2 = nn.Linear(hidden_size, 1)


    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=drop_fc)

  def foward_RNN(self, inputs):
    notes, duree, time, veloc = inputs

    # Embedding layer
    embeddings_note = self.embedding_note(notes) # Output shape (batch, sequence_length, embedding_dim)
    embeddings_duree = self.embedding_duree(duree)
    embeddings_time = self.embedding_time(time)

    emeddings_veloc = self.fc_emb_veloc(veloc)
    #emeddings_veloc =  torch.unsqueeze(emeddings_veloc, 2)

    x = torch.cat((embeddings_note, embeddings_duree, embeddings_time, emeddings_veloc), dim=2)

    # Set initial hidden and cell states 
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
    
    # Forward propagate LSTM
    out, hidden = self.rnn(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
    out = out[:, -1, :] # Hidden state of the last element of the sequence (equivalent to hidden[0])

    return out

  def classif_note(self, out):
    note = self.fc_note_1(out)
    note = self.dropout(note)
    note = self.relu(note)
    note = self.fc_note_2(note)
    return note

  def classif_duree(self, out, note_target):

    embeddings_note_target = self.embedding_note_cond(note_target)

    duree = torch.cat((out, embeddings_note_target), dim=1)
    duree = self.fc_duree_1(duree)
    duree = self.dropout(duree)
    duree = self.relu(duree)
    duree = self.fc_duree_2(duree)
    return duree

  def classif_time(self, out, note_target, duree_target):

    embeddings_note_target = self.embedding_note_cond(note_target)
    embeddings_duree_target = self.embedding_duree_cond(duree_target)

    time = torch.cat((out, embeddings_note_target, embeddings_duree_target), dim=1)
    time = self.fc_time_1(time)
    time = self.dropout(time)
    time = self.relu(time)
    time = self.fc_time_2(time)
    return time

  def reg_veloc(self, out, note_target, duree_target, time_target):

    embeddings_note_target = self.embedding_note_cond(note_target)
    embeddings_duree_target = self.embedding_duree_cond(duree_target)
    embeddings_time_target = self.embedding_time_cond(time_target)

    veloc = torch.cat((out, embeddings_note_target, embeddings_duree_target, embeddings_time_target), dim=1)
    veloc = self.fc_veloc_1(veloc)
    veloc = self.dropout(veloc)
    veloc = self.relu(veloc)
    veloc = self.fc_veloc_2(veloc)
    return veloc


  def forward(self, inputs, targets):
    out = self.foward_RNN(inputs)

    note_target, duree_target, time_target = targets

    note = self.classif_note(out)
    duree = self.classif_duree(out, note_target)
    time = self.classif_time(out, note_target, duree_target)
    veloc = self.reg_veloc(out, note_target, duree_target, time_target)

    return note, duree, time, veloc

# **Training**

## Accuracy 

In [41]:
def validate_model(model, loader):
    model.eval()
    with torch.no_grad():
        correct_note = 0
        correct_duree = 0
        correct_time = 0
        sum_distance = 0

        total = 0
        count = 0
        for batch in loader:
            count += 1
            batch = torch.reshape(batch, (batch.shape[0], -1 , 4))

            notes = batch[:,:,0]
            velocity = batch[:,:,1]
            duree = batch[:,:,2]
            time = batch[:,:,3]

            notes_sequence = notes[:,:16].to(device)
            notes_target = notes[:,16].to(device)

            duree = (duree * 2) // 60 - (duree // 60) - 1
            duree = torch.clip(duree, 0, 31)
            duree_sequence = duree[:,:16].to(device)
            duree_target = duree[:,16].to(device)

            time = (time * 2) // 60 - (time // 60)
            time = torch.clip(time, 0, 17)
            time_sequence = time[:,:16].to(device)
            time_target = time[:,16].to(device)

            velocity_sequence = velocity[:,:16].to(device) / 100
            velocity_sequence = torch.unsqueeze(velocity_sequence, 2)
            velocity_target = velocity[:,16].to(device) / 100

            inputs = (notes_sequence, duree_sequence, time_sequence, velocity_sequence)
            targets = (notes_target, duree_target, time_target)

            note, duree, time, veloc = model(inputs, targets)

            # Accuracy note
            _, predicted = torch.max(note.data, 1)
            correct_note += (predicted == notes_target).sum().item()
            total += notes_target.size(0)

            # Accuracy duree 
            _, predicted = torch.max(duree.data, 1)
            correct_duree += (predicted == duree_target).sum().item()

            # Accuracy time
            _, predicted = torch.max(time.data, 1)
            correct_time += (predicted == time_target).sum().item()

            # Distance velocity
            distance = torch.mean(torch.abs(veloc.squeeze() - velocity_target)) * 100
            sum_distance += distance.item()



        accuracy_note = 100 * correct_note / total
        accuracy_duree = 100 * correct_duree / total
        accuracy_time = 100 * correct_time / total
        distance_velo = sum_distance / count

    return (accuracy_note, accuracy_duree, accuracy_time, distance_velo)

## Training loop

In [42]:
def train_model(model, optimizer, train_loader, val_loader, num_epochs, lr_scheduler=None, display_loss=False):
  criterion_note = nn.CrossEntropyLoss()
  criterion_duree = nn.CrossEntropyLoss()
  criterion_time = nn.CrossEntropyLoss()

  best_val_accuracy_note = 0
  best_epoch_note = 0

  best_val_accuracy_duree = 0
  best_epoch_duree = 0

  best_val_accuracy_time = 0
  best_epoch_time = 0

  best_val_distance_velo = 1000
  best_epoch_velo = 0

  for epoch in range(num_epochs):

    model.train()

    #### UPDATE LEARNING RATE #### 
    if lr_scheduler == 'multi_steps':
        if epoch in [int(num_epochs * 0.5)]:
            for param_group in optimizer.param_groups:
                param_group['lr'] *= 0.1

    for i, batch in enumerate(train_loader):
      batch = torch.reshape(batch, (batch.shape[0], -1 , 4))

      notes = batch[:,:,0]
      velocity = batch[:,:,1]
      duree = batch[:,:,2]
      time = batch[:,:,3]

      notes_sequence = notes[:,:16].to(device)
      notes_target = notes[:,16].to(device)

      duree = (duree * 2) // 60 - (duree // 60) - 1
      duree = torch.clip(duree, 0, 31)
      duree_sequence = duree[:,:16].to(device)
      duree_target = duree[:,16].to(device)

      time = (time * 2) // 60 - (time // 60)
      time = torch.clip(time, 0, 17)
      time_sequence = time[:,:16].to(device)
      time_target = time[:,16].to(device)

      velocity_sequence = velocity[:,:16].to(device) / 100
      velocity_sequence = torch.unsqueeze(velocity_sequence, 2)
      velocity_target = velocity[:,16].to(device) / 100

      inputs = (notes_sequence, duree_sequence, time_sequence, velocity_sequence)
      targets = (notes_target, duree_target, time_target)

      optimizer.zero_grad()
      note, duree, time, veloc = model(inputs, targets)

      loss_note = torch.mean(criterion_note(note, notes_target))
      loss_duree = torch.mean(criterion_duree(duree, duree_target))
      loss_time = torch.mean(criterion_time(time, time_target))
      loss_veloc = torch.mean(torch.abs(veloc.squeeze() - velocity_target))
      loss = loss_note + loss_duree + loss_time + loss_veloc

      loss.backward()
      optimizer.step()

      if i % 300 == 0 and display_loss:
        print(f'Epoch : {epoch}, Step: {i}, Loss: {round(loss.item(), 2)}')

    # Train accuracy 
    train_accuracy_note, train_accuracy_duree, train_accuracy_time, train_distance_velo = validate_model(model, train_loader)
    train_accuracy_note, train_accuracy_duree, train_accuracy_time, train_distance_velo = round(train_accuracy_note, 2), round(train_accuracy_duree, 2), round(train_accuracy_time, 2), round(train_distance_velo, 2)

    # Val accuracy
    val_accuracy_note, val_accuracy_duree, val_accuracy_time, val_distance_velo = validate_model(model, val_loader)
    val_accuracy_note, val_accuracy_duree, val_accuracy_time, val_distance_velo = round(val_accuracy_note, 2), round(val_accuracy_duree, 2), round(val_accuracy_time, 2), round(val_distance_velo, 2)

    if val_accuracy_note > best_val_accuracy_note:
      best_val_accuracy_note = val_accuracy_note
      best_epoch_note = epoch

    if val_accuracy_duree > best_val_accuracy_duree:
      best_val_accuracy_duree = val_accuracy_duree
      best_epoch_duree = epoch

    if val_accuracy_time > best_val_accuracy_time:
      best_val_accuracy_time = val_accuracy_time
      best_epoch_time = epoch

    if val_distance_velo < best_val_distance_velo:
      best_val_distance_velo = val_distance_velo
      best_epoch_velo = epoch

    print('################')
    print(f'Epoch: {epoch}, Loss note: {round(loss_note.item(), 2)}, Loss note on: {round(loss_duree.item(), 2)}, Loss time: {round(loss_time.item(), 2)}, Loss velocity: {round(100 * loss_veloc.item(), 2)}')
    print('------')
    print(f'Epoch : {epoch}, Train accuracy note : {train_accuracy_note} %, Val accuracy note : {val_accuracy_note} %')
    print(f'Best val accuracy at epoch {best_epoch_note}: {best_val_accuracy_note} %')
    print('------')
    print(f'Epoch : {epoch}, Train accuracy duree : {train_accuracy_duree} %, Val accuracy duree : {val_accuracy_duree} %')
    print(f'Best val accuracy at epoch {best_epoch_duree}: {best_val_accuracy_duree} %')
    print('------')
    print(f'Epoch : {epoch}, Train accuracy time : {train_accuracy_time} %, Val accuracy time: {val_accuracy_time} %')
    print(f'Best val accuracy at epoch {best_epoch_time}: {best_val_accuracy_time} %')
    print('------')
    print(f'Epoch : {epoch}, Train distance velo : {train_distance_velo}, Val distance velo: {val_distance_velo}')
    print(f'Best val distance at epoch {best_epoch_velo}: {best_val_distance_velo}')

# **Experiments**

## RNN

In [ ]:
model = RNN(num_classes_note, num_classes_duree, num_classes_time, embedding_dim_note=128, embedding_dim_duree=128, embedding_dim_time=128, embedding_dim_veloc=128, hidden_size=512, num_layers=3).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, nesterov=True, momentum=0.9)

train_model(model, optimizer, train_loader, val_loader, num_epochs=100, lr_scheduler='multi_steps')

## GRU

In [ ]:
model = GRU(num_classes_note, num_classes_duree, num_classes_time, embedding_dim_note=128, embedding_dim_duree=128, embedding_dim_time=128, embedding_dim_veloc=128, hidden_size=512, num_layers=3).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, nesterov=True, momentum=0.9)

train_model(model, optimizer, train_loader, val_loader, num_epochs=100, lr_scheduler='multi_steps')

## LSTM

In [ ]:
model = LSTM(num_classes_note, num_classes_duree, num_classes_time, embedding_dim_note=128, embedding_dim_duree=128, embedding_dim_time=128, embedding_dim_veloc=128, hidden_size=512, num_layers=3).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, nesterov=True, momentum=0.9)

train_model(model, optimizer, train_loader, val_loader, num_epochs=100, lr_scheduler='multi_steps')

# **Music generation**

## Mido utils

In [45]:
def data_to_track(data):
    track = MidiTrack()
    for values in data:
        note = values[0]
        velocity = values[1]
        time = values[2]
        track.append(Message('note_on', channel=0, note=note, velocity=velocity, time=time))
    return(track)
    

In [46]:
def generate_inverse_track(data_sequence):
    track_new = MidiTrack()
    
    abs_time = 0
    messages = []
    
    for note_vect in data_sequence:
        abs_time += note_vect[3]
        messages.append([abs_time, note_vect[1], note_vect[0]])
        messages.append([abs_time + note_vect[2], 0, note_vect[0]])
    
    messages.sort()
    
    time_init = 0
    
    messages_2 = []
    for l in messages:
        note = l[2]
        velocity = l[1]
        time = l[0] - time_init
        time_init = l[0]
        track_new.append(Message('note_on', channel=0, note=note, velocity=velocity, time=time))
    
    #mid = MidiFile()
    #mid.tracks.append(track_new)
    #mid.save(path)
    
    return track_new

In [47]:
def save_track(track, path):
    mid = MidiFile()
    mid.tracks.append(track)
    mid.save(path)

## Generation

In [49]:
n_predictions = 1000
# Temperature parameters
temp_note = 1.5
temp_duree = 1.5
temp_time = 1.5

start_velocity = 50
start_note = 64
start_duree = 240
start_time = 0

list_notes = [start_note]
list_duree = [start_duree]
list_times = [start_time]
list_velocity = [start_velocity]

data = [[start_note, start_velocity, start_duree, start_time]]

In [50]:
look_back = 16 # take the 16 last notes as input
model.eval()

for i in range(n_predictions):
  list_notes_input = list_notes[-look_back:]
  list_duree_input = list_duree[-look_back:]
  list_times_input = list_times[-look_back:]
  list_velocity_input = list_velocity[-look_back:]

  notes_input = torch.reshape(torch.tensor(list_notes_input),(1,-1)).to(device)
  duree_input = torch.reshape(torch.tensor(list_duree_input),(1,-1)).to(device)
  times_input = torch.reshape(torch.tensor(list_times_input),(1,-1)).to(device)
  velo_input = torch.reshape(torch.tensor(list_velocity_input),(1,-1)).to(device)

  duree_input = (duree_input * 2) // 60 - (duree_input // 60) - 1
  duree_input = torch.clip(duree_input, 0, 31)

  times_input = (times_input * 2) // 60 - (times_input // 60)
  times_input = torch.clip(times_input, 0, 17)

  velo_input = torch.unsqueeze(velo_input, 2) / 100

  inputs = (notes_input, duree_input, times_input, velo_input)
  out = model.foward_RNN(inputs)

  # Sample note
  note = model.classif_note(out)
  array_proba_note = torch.softmax(note / temp_note, 1).detach().cpu().numpy()[0]
  note_sampled = np.random.choice(range(num_classes_note), p=array_proba_note)

  note_target = torch.tensor([note_sampled]).to(device)

  # Sample duree
  duree = model.classif_duree(out, note_target)
  array_proba_duree = torch.softmax(duree / temp_duree, 1).detach().cpu().numpy()[0]
  duree_sampled = np.random.choice(range(num_classes_duree), p=array_proba_duree)

  duree_target = torch.tensor([duree_sampled]).to(device)

  # Sample time
  time = model.classif_time(out, note_target, duree_target)
  array_proba_time = torch.softmax(time / temp_time, 1).detach().cpu().numpy()[0]
  time_sampled = np.random.choice(range(num_classes_time), p=array_proba_time)

  time_target = torch.tensor([time_sampled]).to(device)

  # Compute velocity
  velocity = model.reg_veloc(out, note_target, duree_target, time_target)
  velocity = torch.clip(100 * velocity, 30, 100)
  
  final_duree = int((duree_sampled + 1) * 60)
  final_time = int(time_sampled * 60)
  final_velocity = int(velocity.item())

  data.append([note_sampled, final_velocity, final_duree, final_time])

  list_notes.append(note_sampled)
  list_duree.append(final_duree)
  list_times.append(final_time)
  list_velocity.append(final_velocity)

In [51]:
track = generate_inverse_track(data)
save_track(track, 'file.mid')